<a href="https://colab.research.google.com/github/kolomichyk/BMSTU-Hack/blob/develop/Diadal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/kolomichyk/BMSTU-Hack/blob/develop/Diadal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Инициализация проекта

In [ ]:
import pandas as pd
from datetime import datetime
import math
import warnings
from seaborn import pairplot

In [ ]:
import os
os.system("pip3 install seaborn")

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


0

In [ ]:
warnings.filterwarnings('ignore', '.*do not.*', )
pd.options.mode.chained_assignment = None

### Предобработка данных

In [ ]:
data_train = pd.read_csv('train.csv')

In [ ]:
data_train.isna().sum()

wagnum                  0
prev_fr_id              0
prev_is_load            0
prev_date_depart        4
prev_date_arrival     248
prev_snd_org_id         0
prev_rsv_org_id         0
prev_distance        1354
snd_st_id               0
rsv_st_id               0
fr_id                   0
is_load                 0
date_depart             0
rod                     0
common_ch               0
vidsobst                0
snd_org_id              0
rsv_org_id              0
distance             1704
prev_freight            1
prev_fr_group           1
freight                 4
fr_group                4
dtype: int64

In [ ]:
data_train = data_train.drop(columns=['rod', 'date_depart'])

In [ ]:
def GetHours(date):
    date_object = datetime.strptime(date[:10], '%Y-%m-%d')
    data_seconds_arr = list(date[11:].split(':'))
    data_low = float(data_seconds_arr[0]) + float(data_seconds_arr[1])/float(60) + float((data_seconds_arr[2]))/float(3600)
    data_high = float(date_object.timestamp())
    return data_high/float(3600) + data_low

In [ ]:
def ConvertDate(data):
    for i in range(data.shape[0]):
        if (type(data['prev_date_depart'][i]) != float and type(data['prev_date_arrival'][i]) != float):
          depart = GetHours(data['prev_date_depart'][i])
          arrival = GetHours(data['prev_date_arrival'][i])
          data['Travel_time'][i] = arrival - depart

In [ ]:
def AddDotw(data):
  for i in range(data.shape[0]):
    if (type(data['prev_date_arrival'][i]) != float):
      date_tmp = datetime.strptime(data['prev_date_arrival'][i][:10], '%Y-%m-%d')
      data['dotw'][i] = datetime.isoweekday(date_tmp)

In [ ]:
def AddMounth(data):
  for i in range(data.shape[0]):
    if (type(data['prev_date_arrival'][i]) != float):
      tmp = data['prev_date_arrival'][i][:10]
      tmp = list(tmp.split('-'))
      data['Mounth'][i] = int(tmp[1])

In [ ]:
data_train['Travel_time'] = [0.0 for i in range(data_train.shape[0])]
data_train['Mounth'] = [0 for i in range(data_train.shape[0])]
data_train['dotw'] = [0 for i in range(data_train.shape[0])]

In [ ]:
data_train.head(10)

,wagnum,prev_fr_id,prev_is_load,prev_date_depart,prev_date_arrival,prev_snd_org_id,prev_rsv_org_id,prev_distance,snd_st_id,rsv_st_id,...,snd_org_id,rsv_org_id,distance,prev_freight,prev_fr_group,freight,fr_group,Travel_time,Mounth,dotw
0,11,1399685,0.0,2022-03-30 22:55:00,2022-04-05 11:07:00,4266126,2279057,4493.0,3609555,4985724,...,2279057.0,9321847.0,269.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,БАНКЕТКИ,Остальные грузы,132.200000,4,2
1,11,292757,1.0,2022-04-07 08:45:00,2022-04-07 18:22:00,2279057,9321847,269.0,4985724,2492408,...,9321847.0,4272291.0,3584.0,БАНКЕТКИ,Остальные грузы,ГАРНИТУРЫ КУХОН,Остальные грузы,9.616667,4,4
2,11,8674472,1.0,2022-04-11 07:30:00,2022-04-20 15:15:00,9321847,4272291,3584.0,2492408,4288665,...,4272291.0,2555202.0,417.0,ГАРНИТУРЫ КУХОН,Остальные грузы,КАРНИЗЫ ДЕРЕВ,Лесные грузы,223.750000,4,3
3,11,8210522,1.0,2022-04-23 16:20:00,2022-04-28 03:43:00,4272291,2555202,417.0,4288665,2440270,...,2327070.0,2327070.0,248.0,КАРНИЗЫ ДЕРЕВ,Лесные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,107.383333,4,4
4,11,1399685,0.0,2022-05-03 14:47:00,2022-05-04 05:11:00,2327070,2327070,248.0,2440270,2440270,...,2327070.0,2487325.0,5.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,ВАТА МЕД,Остальные грузы,14.400000,5,3
5,11,9276341,0.0,2022-05-05 23:02:22,2022-05-05 23:45:42,2327070,2487325,5.0,2440270,2440270,...,2327070.0,7980401.0,5.0,ВАТА МЕД,Остальные грузы,ВАТА МЕД,Остальные грузы,0.722222,5,4
6,11,9276341,0.0,2022-05-06 00:52:38,2022-05-06 06:03:42,2327070,7980401,5.0,2440270,8450101,...,7980401.0,6550555.0,67.0,ВАТА МЕД,Остальные грузы,РИС НЕШЕЛУШ,Хлебные грузы,5.184444,5,5
7,11,2309180,1.0,2022-05-09 00:13:00,2022-05-10 00:54:00,7980401,6550555,67.0,8450101,2440270,...,2327070.0,2327070.0,67.0,РИС НЕШЕЛУШ,Хлебные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,24.683333,5,2
8,11,1399685,0.0,2022-05-13 07:30:00,2022-05-14 07:54:00,2327070,2327070,67.0,2440270,8450101,...,7980401.0,6550555.0,67.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,РИС НЕШЕЛУШ,Хлебные грузы,24.400000,5,6
9,11,2309180,1.0,2022-05-20 15:06:00,2022-05-21 16:57:00,7980401,6550555,67.0,8450101,2440270,...,2327070.0,7980401.0,67.0,РИС НЕШЕЛУШ,Хлебные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,25.850000,5,6


In [ ]:
ConvertDate(data_train)

In [ ]:
AddDotw(data_train)

In [ ]:
AddMounth(data_train)

In [ ]:
data_train.head(10)

,wagnum,prev_fr_id,prev_is_load,prev_date_depart,prev_date_arrival,prev_snd_org_id,prev_rsv_org_id,prev_distance,snd_st_id,rsv_st_id,fr_id,is_load,date_depart,common_ch,vidsobst,snd_org_id,rsv_org_id,distance,prev_freight,prev_fr_group,freight,fr_group,Travel_time,Mounth,dotw
0,11,1399685,0.0,2022-03-30 22:55:00,2022-04-05 11:07:00,4266126,2279057,4493.0,3609555,4985724,292757,1,2022-04-07 08:45:00,9.0,Собственные,2279057,9321847,269.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,БАНКЕТКИ,Остальные грузы,132.200000,0,0
1,11,292757,1.0,2022-04-07 08:45:00,2022-04-07 18:22:00,2279057,9321847,269.0,4985724,2492408,8674472,1,2022-04-11 07:30:00,9.0,Собственные,9321847,4272291,3584.0,БАНКЕТКИ,Остальные грузы,ГАРНИТУРЫ КУХОН,Остальные грузы,9.616667,0,0
2,11,8674472,1.0,2022-04-11 07:30:00,2022-04-20 15:15:00,9321847,4272291,3584.0,2492408,4288665,8210522,1,2022-04-23 16:20:00,9.0,Собственные,4272291,2555202,417.0,ГАРНИТУРЫ КУХОН,Остальные грузы,КАРНИЗЫ ДЕРЕВ,Лесные грузы,223.750000,0,0
3,11,8210522,1.0,2022-04-23 16:20:00,2022-04-28 03:43:00,4272291,2555202,417.0,4288665,2440270,1399685,0,2022-05-03 14:47:00,9.0,Собственные,2327070,2327070,248.0,КАРНИЗЫ ДЕРЕВ,Лесные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,107.383333,0,0
4,11,1399685,0.0,2022-05-03 14:47:00,2022-05-04 05:11:00,2327070,2327070,248.0,2440270,2440270,9276341,0,2022-05-05 23:02:22,9.0,Собственные,2327070,2487325,5.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,ВАТА МЕД,Остальные грузы,14.400000,0,0
5,11,9276341,0.0,2022-05-05 23:02:22,2022-05-05 23:45:42,2327070,2487325,5.0,2440270,2440270,9276341,0,2022-05-06 00:52:38,9.0,Собственные,2327070,7980401,5.0,ВАТА МЕД,Остальные грузы,ВАТА МЕД,Остальные грузы,0.722222,0,0
6,11,9276341,0.0,2022-05-06 00:52:38,2022-05-06 06:03:42,2327070,7980401,5.0,2440270,8450101,2309180,1,2022-05-09 00:13:00,9.0,Собственные,7980401,6550555,67.0,ВАТА МЕД,Остальные грузы,РИС НЕШЕЛУШ,Хлебные грузы,5.184444,0,0
7,11,2309180,1.0,2022-05-09 00:13:00,2022-05-10 00:54:00,7980401,6550555,67.0,8450101,2440270,1399685,0,2022-05-13 07:30:00,9.0,Собственные,2327070,2327070,67.0,РИС НЕШЕЛУШ,Хлебные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,24.683333,0,0
8,11,1399685,0.0,2022-05-13 07:30:00,2022-05-14 07:54:00,2327070,2327070,67.0,2440270,8450101,2309180,1,2022-05-20 15:06:00,9.0,Собственные,7980401,6550555,67.0,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,РИС НЕШЕЛУШ,Хлебные грузы,24.400000,0,0
9,11,2309180,1.0,2022-05-20 15:06:00,2022-05-21 16:57:00,7980401,6550555,67.0,8450101,2440270,1399685,0,2022-05-27 13:11:00,9.0,Собственные,2327070,7980401,67.0,РИС НЕШЕЛУШ,Хлебные грузы,ГАЗЫ УГЛЕВОД ПР,Нефтяные грузы,25.850000,0,0


In [ ]:
def GetAvgMounth(data):
  amount = 0
  sum = 0
  for i in range(data.shape[0]):
    if (data['Mounth'][i] != 0):
        amount += 1
        sum += data['Mounth'][i]
  return sum / amount

In [ ]:
def GetAvgDotw(data):
  amount = 0
  sum = 0
  for i in range(data.shape[0]):
    if (data['dotw'][i] != 0):
        amount += 1
        sum += data['dotw'][i]
  return sum / amount

In [ ]:
def GetAvgTravel_time(data):
  amount = 0
  sum = 0
  for i in range(data.shape[0]):
    if (data['Travel_time'][i] != 0):
        amount += 1
        sum += data['Travel_time'][i]
  return sum / amount

In [ ]:
def GetAvgDistance(data):
  amount = 0
  sum = 0
  for i in range(data.shape[0]):
    if (not math.isnan(data['distance'][i])):
        amount += 1
        sum += data['distance'][i]
  return sum / amount

In [ ]:
def GetAvgPrevDistance(data):
  amount = 0
  sum = 0
  for i in range(data.shape[0]):
    if (not math.isnan(data['prev_distance'][i])):
        amount += 1
        sum += data['prev_distance'][i]
  return sum / amount

In [ ]:
arrAvg = list()
arrAvg.append(GetAvgTravel_time(data_train))
arrAvg.append(GetAvgMounth(data_train))
arrAvg.append(GetAvgDotw(data_train))
arrAvg.append(GetAvgDistance(data_train))
arrAvg.append(GetAvgPrevDistance(data_train))

In [ ]:
arrAvg[1] = int(arrAvg[1])
arrAvg[2] = int(arrAvg[2])

In [ ]:
def FillNa(data, arrAvg):
  for i in range(data.shape[0]):
    if (data['Travel_time'][i] == 0):
      data['Travel_time'][i] = arrAvg[0]
    if (data['Mounth'][i] == 0):
      data['Mounth'][i] = arrAvg[1]
    if (data['dotw'][i] == 0):
      data['dotw'][i] == arrAvg[2]
    if (math.isnan(data['distance'][i])):
      data['distance'][i] = arrAvg[3]
    if (math.isnan(data['prev_distance'][i])): 
      data['prev_distance'][i] = arrAvg[3]

In [ ]:
FillNa(data_train, arrAvg)

### Выгрузка обработанных данных

In [ ]:
data_train = data_train.drop(columns=['prev_date_depart', 'prev_date_arrival'])